In [1]:
from yargy import Parser, rule, and_, or_
from yargy.predicates import gram, is_capitalized, dictionary
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from yargy.relations import gnc_relation

In [2]:
Name = fact(
    "Name",
    ["first", "last"]
)

BirthDate = fact(
    "BirthDate",
    ["date"]
)

BirthPlace = fact(
    "BirthPlace",
    ["in_", "place"]
)

Entry = fact(
    "Entry",
    ["name", "birth_date", "birth_place"]
)

# Паттерн для даты

In [3]:
from yargy.predicates import gte, lte
from yargy.predicates import caseless, normalized, dictionary


MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'мая',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}
YEAR_WORDS = or_(
    rule(caseless('г'), '.'),
    rule(normalized('год'))
)
Date = fact(
    'Date',
    ['year', 'month', 'day']
)

DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day
)
MONTH = and_(
    gte(1),
    lte(12)
).interpretation(
    Date.month
)
YEAR = and_(
    gte(1),
    lte(2018)
).interpretation(
    Date.year
)
MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month
)
DATE = rule(
    normalized("в").optional(),
    DAY.optional(),
    MONTH_NAME.optional(),
    YEAR,
    YEAR_WORDS.optional()
).interpretation(Date)

# Паттерн для имени

In [4]:
gnc = gnc_relation()

NAME = rule(
    is_capitalized().interpretation(Name.first.inflected()).optional(),
    is_capitalized().interpretation(Name.last.inflected()),
).interpretation(
    Name
)

In [5]:
parser = Parser(NAME)
text = "Трэмиел родился в 1928 году в Польше"

for match in parser.findall(text):
    print(match.fact)

Name(first=None, last='трэмиел')
Name(first=None, last='польша')


# Паттерн для места рождения

In [6]:
PLACE = rule(
    rule(normalized("в")).interpretation(BirthPlace.in_),
    rule(is_capitalized()).interpretation(BirthPlace.place),
).interpretation(
    BirthPlace
)

parser = Parser(PLACE)
text = "Трэмиел родился в 1928 году в Польше"

for match in parser.findall(text):
    print(match.fact)

BirthPlace(in_='в', place='Польше')


# Общий паттерн

In [11]:
PlaceDate = fact(
    "PlaceDate",
    ["place", "date"]
)

DatePlace = fact(
    "DatePlace",
    ["date", "place"]
)

RULE = rule(
    NAME.interpretation(Entry.name),
    normalized("родился"),
    or_(
        rule(
            PLACE.interpretation(Entry.birth_place),
            DATE.interpretation(Entry.birth_date)
        ).interpretation(PlaceDate),
        rule(
            DATE.interpretation(Entry.birth_date),
            PLACE.interpretation(Entry.birth_place)
        ).interpretation(DatePlace),
    )
).interpretation(Entry)

parser = Parser(RULE)
text = "Трэмиел родился в 1928 году в Польше"

for match in parser.findall(text):
    print(match.fact)
    print([_.value for _ in match.tokens])

Entry(name=Name(first=None, last='трэмиел'), birth_date=None, birth_place=None)
['Трэмиел', 'родился', 'в', '1928', 'году', 'в', 'Польше']


In [12]:
PlaceDate = fact(
    "PlaceDate",
    ["place", "date"]
)

DatePlace = fact(
    "DatePlace",
    ["date", "place"]
)


RULE = rule(
    NAME.interpretation(Entry.name),
    caseless("родился"),
    or_(
        rule(
            PLACE.interpretation(Entry.birth_place),
            DATE.interpretation(Entry.birth_date)
        ),
        rule(
            DATE.interpretation(Entry.birth_date),
            PLACE.interpretation(Entry.birth_place)
        ),
    )
).interpretation(Entry)

parser = Parser(RULE)
text = "Трэмиел родился в 1928 году в Польше\nОн родился во Франции 11 сентября 1865 года\nЮрий Яковлев родился в Москве в 1928 году"

for t in text.split("\n"):
    for match in parser.findall(t):
        print('-' * 82)
        print(match.fact.name)
        print(match.fact.birth_date)
        print(match.fact.birth_place)

----------------------------------------------------------------------------------
Name(first=None, last='трэмиел')
Date(year='1928', month=None, day=None)
BirthPlace(in_='в', place='Польше')
----------------------------------------------------------------------------------
Name(first=None, last='он')
Date(year='1865', month='сентября', day='11')
BirthPlace(in_='во', place='Франции')
----------------------------------------------------------------------------------
Name(first='юрий', last='яковлев')
Date(year='1928', month=None, day=None)
BirthPlace(in_='в', place='Москве')


# Парсинг из всех данных

In [1]:
import gzip

from dataclasses import dataclass
from typing import Iterator

@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

texts = list(read_texts("../data/news.tar.gz"))

In [16]:
from tqdm.notebook import tqdm

matches = []

for text in tqdm(texts):
    for match in parser.findall(text.text):
        print(match.fact)
        matches.append((text, match))

  0%|          | 0/10000 [00:00<?, ?it/s]

Entry(name=Name(first=None, last='трэмиел'), birth_date=Date(year='1928', month=None, day=None), birth_place=BirthPlace(in_='в', place='Польше'))
Entry(name=Name(first=None, last='он'), birth_date=Date(year='1865', month='сентября', day='11'), birth_place=BirthPlace(in_='в', place='Польше'))
Entry(name=Name(first=None, last='ребёнок'), birth_date=Date(year='25', month=None, day=None), birth_place=BirthPlace(in_='в', place='Калифорнии'))
Entry(name=Name(first='дмитрий', last='чернявский'), birth_date=Date(year='1992', month='марта', day='5'), birth_place=BirthPlace(in_='в', place='Артемовске'))
Entry(name=Name(first=None, last='миллиардер'), birth_date=Date(year='1938', month=None, day=None), birth_place=BirthPlace(in_='в', place='Лондоне'))
Entry(name=Name(first='яковлевюрий', last='яковлев'), birth_date=Date(year='1928', month=None, day=None), birth_place=BirthPlace(in_='в', place='Москве'))
Entry(name=Name(first=None, last='патрик'), birth_date=Date(year='1990', month=None, day=None)

Посмотрим с контекстом

In [41]:
for item in matches:
    text = item[0]
    match = item[1]
    left, right = match.tokens[0].span.start, match.tokens[-1].span.stop
    print(f"Extracted from: {text.text[left:right]}\nmatched: {match.fact}")
    print('-' * 80)

Extracted from: Трэмиел родился в 1928 году в Польше
matched: Entry(name=Name(first=None, last='трэмиел'), birth_date=Date(year='1928', month=None, day=None), birth_place=BirthPlace(in_='в', place='Польше'))
--------------------------------------------------------------------------------
Extracted from: Он родился в Польше 11 сентября 1865 года
matched: Entry(name=Name(first=None, last='он'), birth_date=Date(year='1865', month='сентября', day='11'), birth_place=BirthPlace(in_='в', place='Польше'))
--------------------------------------------------------------------------------
Extracted from: Ребенок родился в Калифорнии 25
matched: Entry(name=Name(first=None, last='ребёнок'), birth_date=Date(year='25', month=None, day=None), birth_place=BirthPlace(in_='в', place='Калифорнии'))
--------------------------------------------------------------------------------
Extracted from: Дмитрий Чернявский родился в Артемовске 5 марта 1992 года
matched: Entry(name=Name(first='дмитрий', last='чернявск

В целом проблем почти нет